In [1]:
from pathlib import Path

BASE = Path("/Users/joregan/Playing/rd_ctm_edit")

In [11]:
files = []
with open("/tmp/terror-files") as terrorf:
    for ff in terrorf.readlines():
        files.append(ff.strip())

In [50]:
def tidy(word, lc=True):
    if word[0:1] == '"':
        word = word[1:]
    if word[-1:] == '"':
        word = word[:-1]
    if word[-1:] in ".!?,;-":
        word = word[:-1]
    if lc:
        return word.lower()
    else:
        return word

In [20]:
import difflib

a = "afkanistan"
b = "Afghanistan."
def ratio(a, b):
    return difflib.SequenceMatcher(None, a, tidy(b)).ratio()

print(ratio(a, b))
print(ratio("male", "Mali"))
print(ratio("normader", "Nomader"))

0.8571428571428571
0.75
0.9333333333333333


In [22]:
def slurp(filename):
    lines = []
    with open(filename) as infile:
        for line in infile.readlines():
            lines.append(line.strip())
    return lines

In [23]:
testing = slurp("/Users/joregan/Playing/rd_ctm_edit/H001UFöU4")

In [24]:
testing[0:5]

['2442207150019781021 1 43.44 0.02 k 1.0 Jag sub',
 '2442207150019781021 1 49.04 0.42 härtalman 1.0 yrkar sub',
 '2442207150019781021 1 50.2 0.059 jag 1.0 bifall sub',
 '2442207150019781021 1 50.34 0.379 bifaller 1.0 till sub',
 '2442207150019781021 1 50.78 0.659 reservationen. 1.0 reservationen. cor']

In [25]:
CONJ = ["och", "men"]

In [54]:
def modify_pairs(sent_a, sent_b):
    a = sent_a.split(" ")
    b = sent_b.split(" ")

    print(a[4], a[6], b[4], b[6])
    if a[4] == tidy(b[6]) or is_subst(a[4], tidy(b[6], False)):
        if a[6] == "<eps>":
            a[4] = a[6] = b[6]
            b[6] = "<eps>"
            a[7] = "cor"
            b[7] = "ins"
        if b[4] in CONJ:
            b[7] = "ins-conj"
    if a[4] == "<eps>":
        if a[6] + b[6] == b[4]:
            joined = f"{a[6]}_{b[6]}"
            b[4] = b[6] = joined
            b[7] = "cor"
            a = []
    
    return (" ".join(a), " ".join(b))

In [55]:
modify_pairs("2442207150019781021 1 58.4 0.78 invandringspolitiken 1.0 <eps> ins",
             "2442207150019781021 1 59.9 0.079 men 1.0 invandringspolitiken. sub")

invandringspolitiken <eps> men invandringspolitiken.


('2442207150019781021 1 58.4 0.78 invandringspolitiken. 1.0 invandringspolitiken. cor',
 '2442207150019781021 1 59.9 0.079 men 1.0 <eps> ins-conj')

In [56]:
modify_pairs("2442207150019781021 1 97.12 0.0 <eps> 1.0 insats del",
             "2442207150019781021 1 97.18 0.739 insatsregeringen 1.0 regeringen sub")

<eps> insats insatsregeringen regeringen


('',
 '2442207150019781021 1 97.18 0.739 insats_regeringen 1.0 insats_regeringen cor')

In [57]:
modify_pairs("2442207150019781021 1 121.18 0.559 afkanistan 1.0 <eps> ins",
"2442207150019781021 1 122.62 0.079 och 1.0 Afghanistan. sub")

afkanistan <eps> och Afghanistan.


('2442207150019781021 1 121.18 0.559 Afghanistan. 1.0 Afghanistan. cor',
 '2442207150019781021 1 122.62 0.079 och 1.0 <eps> ins-conj')

In [61]:
_SUBS = """
afkanistan Afghanistan
maly Mali
male Mali
övvik Ö-vik
abebégymnasiet ABB-gymnasiet
abeflygplan A/B-flygplan
abefs ABF:s
abfs ABF:s
abies ABS
abieffkurserna ABF-kursern
abief ABF
aczen Axén
adeärrkort ADR-kort
adeärr ADR
adiellbedömningen ADL-bedömningen
adiesselstationer ADSL-stationer
adihodepreparaten adhd-preparat
adihodemediciner adhd-mediciner
adihodeläkemedel adhd-läkemedel
adihodedroger adhd-droger
aduptioner adoptioner
aeff AF
aelless ALS
aenddeesstrategin ANDTS-strategin
aettaffisch A1-affisch
aendete ANDT
aendeteanvändandet ANDT-användandet
aendeteessfrågorna ANDTS-frågorna
aendeteesspolitiken ANDTS-politiken
aendeteessstrategin ANDTS-strategin
aendetefrågan ANDT-frågan
aendetefrågor ANDT-frågor
aendeteområdet ANDT-området
aendetessområdet ANDTS-området
aendetesspolitiskt ANDTS-politiskt
aendetesstrategi ANDTS-strategi
aendetestrategi ANDT-strategi
aendetestrategin ANDT-strategin
rutavdraget RUT-avdraget
"""
def get_subs():
    subs = {}
    for l in _SUBS.split("\n"):
        if l.strip() == "":
            continue
        p = l.strip().split(" ")
        if not p[0] in subs:
            subs[p[0]] = []
        for part in p[1:]:
            subs[p[0]].append(part)
    return subs

In [62]:
SUBS = get_subs()

In [45]:
def is_subst(a, b):
    if a in SUBS:
        if b in SUBS[a]:
            return True
    return False

In [46]:
is_subst("male", "Mali")

True

In [ ]:
def partition(lines):
    def has_final(line):
        FINAL = ".!?"
        parts = line.strip().split(" ")
        piece = parts[6]
        if piece.endswith('"'):
            piece = piece[:-1]
        return piece[-1:] in FINAL
    
    def is_capital(line):
        parts = line.strip().split(" ")
        piece = parts[6]
        return piece[0:1].isupper()
    
    def splittable(a, b):
        return has_final(a) and is_capital(b)

    sentences = []
    current = []

    for line in lines:
        pass     

In [12]:
OUTPATH = Path("/tmp/outfiles")
for file in files:
    OUTFILE = OUTPATH / file.stem
    with open(OUTFILE, "w") as outputfile:
        pass
    

['/Users/joregan/Playing/rd_ctm_edit/H001FöU6',
 '/Users/joregan/Playing/rd_ctm_edit/H001KU20',
 '/Users/joregan/Playing/rd_ctm_edit/H001SoU15',
 '/Users/joregan/Playing/rd_ctm_edit/H001UFöU4',
 '/Users/joregan/Playing/rd_ctm_edit/H010318',
 '/Users/joregan/Playing/rd_ctm_edit/H010330',
 '/Users/joregan/Playing/rd_ctm_edit/H010423',
 '/Users/joregan/Playing/rd_ctm_edit/H101FöU11',
 '/Users/joregan/Playing/rd_ctm_edit/H101FöU5',
 '/Users/joregan/Playing/rd_ctm_edit/H101FöU6',
 '/Users/joregan/Playing/rd_ctm_edit/H101FöU7',
 '/Users/joregan/Playing/rd_ctm_edit/H101JuU11',
 '/Users/joregan/Playing/rd_ctm_edit/H101JuU12',
 '/Users/joregan/Playing/rd_ctm_edit/H101JuU25',
 '/Users/joregan/Playing/rd_ctm_edit/H101JuU39',
 '/Users/joregan/Playing/rd_ctm_edit/H101KU14',
 '/Users/joregan/Playing/rd_ctm_edit/H101UU16',
 '/Users/joregan/Playing/rd_ctm_edit/H101UU18',
 '/Users/joregan/Playing/rd_ctm_edit/H101UU25',
 '/Users/joregan/Playing/rd_ctm_edit/H101UU8',
 '/Users/joregan/Playing/rd_ctm_edit/